In [1]:
sc.stop()
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

conf = SparkConf().setAppName("一打code就想睡")\
                  .setMaster("spark://10.120.26.200:7077")\
                  .set("spark.executor.memory", '12g')\
                  .set('spark.cores.max', '4')

sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName('一打code就想睡').getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
from pyspark.sql.types import *
from pyspark.sql import functions as F

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
schema = StructType([StructField("Date",DateType(),True),StructField("Open",FloatType(),True),StructField("High",FloatType(),True),StructField("Low",FloatType(),True),StructField("Close",FloatType(),True),StructField("Adj Close",FloatType(),True),StructField("Volume",IntegerType(),True)])
data = spark.read.csv("hdfs://10.120.26.200/user/spark/spark101/spark203/1101.TW.csv", header=True, schema = schema)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
data.count()
data.show()
data.na.drop().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+-------+-------+-------+---------+---------+
|      Date|   Open|   High|    Low|  Close|Adj Close|   Volume|
+----------+-------+-------+-------+-------+---------+---------+
|      null|   null|   null|   null|   null|     null|     null|
|      null|   null|   null|   null|   null|     null|     null|
|      null|   null|   null|   null|   null|     null|     null|
|      null|   null|   null|   null|   null|     null|     null|
|      null|   null|   null|   null|   null|     null|     null|
|      null|   null|   null|   null|   null|     null|     null|
|      null|   null|   null|   null|   null|     null|     null|
|      null|   null|   null|   null|   null|     null|     null|
|2000-01-14|18.0688|18.2268|17.3839|17.5946| 6.985693| 38590181|
|2000-01-17|17.9634|18.5429|17.7527|18.2268| 7.236699| 66482427|
|2000-01-18|18.2268|18.2268|   17.7|17.7527| 7.048464| 29812552|
|      null|   null|   null|   null|   null|     null|     null|
|2000-01-20|18.0161| 19.0

In [23]:
data = data.na.drop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
df = data.filter("Volume != 0")
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+-------+-------+-------+---------+---------+
|      Date|   Open|   High|    Low|  Close|Adj Close|   Volume|
+----------+-------+-------+-------+-------+---------+---------+
|2000-01-14|18.0688|18.2268|17.3839|17.5946| 6.985693| 38590181|
|2000-01-17|17.9634|18.5429|17.7527|18.2268| 7.236699| 66482427|
|2000-01-18|18.2268|18.2268|   17.7|17.7527| 7.048464| 29812552|
|2000-01-20|18.0161| 19.017|18.0161| 19.017| 7.550436|172525505|
|2000-01-25| 20.492|20.7554|20.0705| 20.492| 8.136064| 72595917|
|2000-01-28|  20.65|20.7027|19.7545|20.0179| 7.947831| 59100205|
|2000-01-31|20.0179|20.4393|20.0179|20.1759| 8.010561| 44063449|
|2000-02-14|24.7589|24.7589|   23.6|23.9688|  9.51648| 69766437|
|2000-02-16|25.6545|26.3393|25.2857|26.3393|10.457655| 78954763|
|2000-02-21|27.3929|27.6563|26.6027|26.6027|10.562234|122738687|
|2000-02-22|25.1804|25.7071|24.7589|24.7589| 9.830177| 43635836|
|2000-02-25|24.3902|24.3902|22.8098|22.8098| 9.056314| 73377468|
|2000-02-29|22.8098|23.12

In [25]:
df.createOrReplaceTempView("stock")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
l="""
SELECT 
    Date,
    Open,
    Close,
    Low,
    High,
    MIN(Low) OVER (ORDER BY Date ASC ROWS 8 PRECEDING) AS min,
    MAX(High) OVER (ORDER BY Date ASC ROWS 8 PRECEDING) AS max,
    (Close-MIN(Low) OVER (ORDER BY Date ASC ROWS 8 PRECEDING))/(MAX(High) OVER (ORDER BY Date ASC ROWS 8 PRECEDING)-MIN(Low) OVER (ORDER BY Date ASC ROWS 8 PRECEDING)) as RSV,
    Volume
FROM   stock
"""

n="""
SELECT Date
FROM stock
"""


rsv_df = spark.sql(l)
rsv_df.show()
rsv_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+-------+-------+-------+-------+-------+--------------------+---------+
|      Date|   Open|  Close|    Low|   High|    min|    max|                 RSV|   Volume|
+----------+-------+-------+-------+-------+-------+-------+--------------------+---------+
|2000-01-14|18.0688|17.5946|17.3839|18.2268|17.3839|18.2268| 0.24997114882004332| 38590181|
|2000-01-17|17.9634|18.2268|17.7527|18.5429|17.3839|18.5429|  0.7272656957129927| 66482427|
|2000-01-18|18.2268|17.7527|   17.7|18.2268|17.3839|18.5429|  0.3182062042294084| 29812552|
|2000-01-20|18.0161| 19.017|18.0161| 19.017|17.3839| 19.017|                 1.0|172525505|
|2000-01-25| 20.492| 20.492|20.0705|20.7554|17.3839|20.7554|  0.9218747966918547| 72595917|
|2000-01-28|  20.65|20.0179|19.7545|20.7027|17.3839|20.7554|  0.7812548617165176| 59100205|
|2000-01-31|20.0179|20.1759|20.0179|20.4393|17.3839|20.7554|  0.8281179902887301| 44063449|
|2000-02-14|24.7589|23.9688|   23.6|24.7589|17.3839|24.7589|   0.892867783368644

In [38]:
tdf = rsv_df.withColumn("RSV_t",F.col("RSV")+ 1)
tdf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+-------+-------+-------+-------+-------+--------------------+---------+------------------+
|      Date|   Open|  Close|    Low|   High|    min|    max|                 RSV|   Volume|             RSV_t|
+----------+-------+-------+-------+-------+-------+-------+--------------------+---------+------------------+
|2000-01-14|18.0688|17.5946|17.3839|18.2268|17.3839|18.2268| 0.24997114882004332| 38590181|1.2499711488200433|
|2000-01-17|17.9634|18.2268|17.7527|18.5429|17.3839|18.5429|  0.7272656957129927| 66482427|1.7272656957129926|
|2000-01-18|18.2268|17.7527|   17.7|18.2268|17.3839|18.5429|  0.3182062042294084| 29812552|1.3182062042294085|
|2000-01-20|18.0161| 19.017|18.0161| 19.017|17.3839| 19.017|                 1.0|172525505|               2.0|
|2000-01-25| 20.492| 20.492|20.0705|20.7554|17.3839|20.7554|  0.9218747966918547| 72595917|1.9218747966918546|
|2000-01-28|  20.65|20.0179|19.7545|20.7027|17.3839|20.7554|  0.7812548617165176| 59100205|1.7812548617165176|
|

In [39]:
K = 0
def KV(rsv):
    global K
    K = (2/3) * K + (1/3) * rsv
    return K

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
KV_df = rsv_df.withColumn("KV",KV(F.col("RSV")))
KV_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+-------+-------+-------+-------+-------+--------------------+---------+--------------------+
|      Date|   Open|  Close|    Low|   High|    min|    max|                 RSV|   Volume|                  KV|
+----------+-------+-------+-------+-------+-------+-------+--------------------+---------+--------------------+
|2000-01-14|18.0688|17.5946|17.3839|18.2268|17.3839|18.2268| 0.24997114882004332| 38590181| 0.17590562324373415|
|2000-01-17|17.9634|18.2268|17.7527|18.5429|17.3839|18.5429|  0.7272656957129927| 66482427|  0.5117795636498836|
|2000-01-18|18.2268|17.7527|   17.7|18.2268|17.3839|18.5429|  0.3182062042294084| 29812552|  0.2239228844577318|
|2000-01-20|18.0161| 19.017|18.0161| 19.017|17.3839| 19.017|                 1.0|172525505|  0.7037037037037037|
|2000-01-25| 20.492| 20.492|20.0705|20.7554|17.3839|20.7554|  0.9218747966918547| 72595917|  0.6487267087831569|
|2000-01-28|  20.65|20.0179|19.7545|20.7027|17.3839|20.7554|  0.7812548617165176| 59100205|  0.5